## Limpieza del Dataset

El dataset utilizado es el de online retail, la versión provista para este trabajo. Vamos a hacer una visión general de sus características, ya conocidas desde el anterior parcial.

In [27]:
import pandas as pd
retail_df = pd.read_excel('./data/online_retail_2.xlsx')
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


El dataset cuenta con 8 features que se describen de la siguiente manera:

- InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invoice Date and time. Numeric, the day and time when each transaction was generated.
- UnitPrice: Unit price. Numeric, Product price per unit in sterling.
- CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal, the name of the country where each customer resides.

In [28]:
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [29]:
retail_df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


Vemos que existen precios de unidad negativos e iguales a cero.

In [30]:
retail_df[retail_df['UnitPrice'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


In [31]:
retail_df[retail_df['UnitPrice'] <= 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
536981,581234,72817,NaN,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom
538504,581406,46000M,POLYESTER FILLER PAD 45x45cm,240,2011-12-08 13:58:00,0.0,NaN,United Kingdom
538505,581406,46000S,POLYESTER FILLER PAD 40x40cm,300,2011-12-08 13:58:00,0.0,NaN,United Kingdom
538554,581408,85175,NaN,20,2011-12-08 14:06:00,0.0,NaN,United Kingdom


Los negativos corresponden a ajustes de deuda, que no deberían ser contados. Por otro lado, los productos que no tienen precio unitario tampoco tienen cliente o descripción en la mayoría de los casos. Esto nos da a entender que son transacciones de algún modo extrañas. Como no tenemos el contexto sufieciente para saber que significan estas transacciones preferimos eliminarlas.

Además, para las que tienen descripción, consideramos que si el precio unitario es cero deben haber sido un regalo de la tienda por alguna promoción. Por lo que no sería realista incluirlo en el análisis de itemset frecuentes. No son productos que los clientes hayan comprado, si no que la tienda se los mandó como extra.

In [32]:
retail_df = retail_df[retail_df['UnitPrice'] > 0]

Otra cuestión a tener en cuenta son las devoluciones. Que justamente son las transacciones con cantidades negativas.

In [33]:
devoluciones_df = retail_df[retail_df['Quantity'] < 0]
print("Registros con devolución: ", devoluciones_df.shape[0], "/", retail_df.shape[0])
devoluciones_df.head()

Registros con devolución:  9288 / 539392


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom


Queremos eliminar estas transacciones debido a que pueden ensuciar el análisis de itemsets frecuentes. Nos interesa que compra la gente en conjunto, y agregar transacciones de devolución puede generar que, si alguien compró y canceló la compra, ese conjunto de items se cuente dos veces cuando sólo se dió una vez.

In [34]:
retail_df = retail_df[retail_df['Quantity'] > 0]

In [35]:
retail_df = retail_df[~retail_df["InvoiceNo"].str.contains("C", na=False)]

In [36]:
retail_df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,530104.000000,530104,530104.000000,397884.000000
mean,10.542037,2011-07-04 20:16:05.225087744,3.907625,15294.423453
min,1.000000,2010-12-01 08:26:00,0.001000,12346.000000
25%,1.000000,2011-03-28 12:22:00,1.250000,13969.000000
50%,3.000000,2011-07-20 12:58:00,2.080000,15159.000000
75%,10.000000,2011-10-19 12:39:00,4.130000,16795.000000
max,80995.000000,2011-12-09 12:50:00,13541.330000,18287.000000
std,155.524124,NaN,35.915681,1713.141560


Vamos a analizar los valores nulos.

In [37]:
retail_df.isnull().sum()

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     132220
Country             0
dtype: int64

Veamos las transacciones sin clientes.

In [38]:
without_client = retail_df[retail_df['CustomerID'].isnull()]
without_client.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom
1447,536544,21790,VINTAGE SNAP CARDS,9,2010-12-01 14:32:00,1.66,NaN,United Kingdom


In [39]:
without_client.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,132220.000000,132220,132220.000000,0.0
mean,3.180790,2011-06-16 08:38:17.349720064,6.288362,NaN
min,1.000000,2010-12-01 14:32:00,0.060000,NaN
25%,1.000000,2011-02-18 16:12:00,1.630000,NaN
50%,1.000000,2011-06-28 15:44:00,3.290000,NaN
75%,3.000000,2011-10-12 16:36:00,5.790000,NaN
max,1820.000000,2011-12-09 10:26:00,13541.330000,NaN
std,11.199162,NaN,60.783941,NaN


Existen muchos CustomerID nulos y algunas cuantas descripciones. A diferencia del clustering de la vez pasada, no nos importa quien compra sino que compra. Por lo que en este caso no vemos necesario eliminar las transacciones que no tengan CustomerID, ya que haremos la agrupación por factura. Los valores nulos son, seguramente, resultado de la eliminación del cliente en la base de datos, pero parecen ser transacciones válidas.

Vamos a probar eliminar repetidos.

In [40]:
print('Instancias duplicadas: {}'.format(retail_df.duplicated().sum()))
retail_df = retail_df.drop_duplicates()
print('Instancias duplicadas: {}'.format(retail_df.duplicated().sum()))

Instancias duplicadas: 5226
Instancias duplicadas: 0


Finalmente, vamos a analizar los productos que no tienen un stockcode acorde al formato de seis dígitos que se menciona en la información del dataset.

In [41]:
retail_df['StockCode'] = retail_df['StockCode'].astype(str)
anomaly_stock_codes = retail_df[(retail_df['StockCode']).str.contains('^[a-zA-Z]',regex=True)]['StockCode']
print(f"Cantidad de StockCode que no cumplen con el formato: {anomaly_stock_codes.nunique()} \n")
for stock_code in anomaly_stock_codes.unique():
    desc = retail_df[retail_df['StockCode'] == stock_code]
    print(f"{stock_code} ---> {desc['Description'].iloc[0]}")
print(f"\nInstancias de StockCode que no cumplen con el formato: {anomaly_stock_codes.shape[0]}")

Cantidad de StockCode que no cumplen con el formato: 22 

POST ---> POSTAGE
C2 ---> CARRIAGE
DOT ---> DOTCOM POSTAGE
M ---> Manual
BANK CHARGES ---> Bank Charges
AMAZONFEE ---> AMAZON FEE
DCGS0076 ---> SUNJAR LED NIGHT NIGHT LIGHT
DCGS0003 ---> BOXED GLASS ASHTRAY
gift_0001_40 ---> Dotcomgiftshop Gift Voucher £40.00
DCGS0070 ---> CAMOUFLAGE DOG COLLAR
m ---> Manual
gift_0001_50 ---> Dotcomgiftshop Gift Voucher £50.00
gift_0001_30 ---> Dotcomgiftshop Gift Voucher £30.00
gift_0001_20 ---> Dotcomgiftshop Gift Voucher £20.00
DCGS0069 ---> OOH LA LA DOGS COLLAR
DCGSSBOY ---> BOYS PARTY BAG
DCGSSGIRL ---> GIRLS PARTY BAG
gift_0001_10 ---> Dotcomgiftshop Gift Voucher £10.00
S ---> SAMPLES
PADS ---> PADS TO MATCH ALL CUSHIONS
DCGS0004 ---> HAYNES CAMPER SHOULDER BAG
B ---> Adjust bad debt

Instancias de StockCode que no cumplen con el formato: 2374


Queremos eliminar aquellos que no son productos reales, si no tasas de servicios o impuestos. Pero existen otros que si parecen ser productos pero no cumplen el requisito. Vamos a analizar la cantidad de cada codigo para ver que sucede con ellos.

In [42]:
retail_df[retail_df['StockCode'].isin(anomaly_stock_codes)]['StockCode'].value_counts()

StockCode
POST            1126
DOT              706
M                316
C2               141
DCGSSGIRL         13
BANK CHARGES      12
DCGSSBOY          11
gift_0001_20       9
gift_0001_10       8
gift_0001_30       7
gift_0001_50       4
DCGS0003           4
PADS               3
gift_0001_40       3
AMAZONFEE          2
S                  2
DCGS0076           2
DCGS0070           1
m                  1
DCGS0069           1
DCGS0004           1
B                  1
Name: count, dtype: int64

Vemos que la mayoría aparecen muy pocas veces en el dataset, por lo que es probable que no afecten demasiado al análisis. Vamos a dejar los que parecen productos y eliminar solamente tasas de servicios o impuestos.

In [43]:
retail_df = retail_df[~retail_df['StockCode'].isin(['POST', 'C2', 'DOT', 'M', 'm', 'BANK CHARGES', 'AMAZONFEE', 'S', 'PADS', 'B'])]

## Creación del dataset para Itemsets

Para los algoritmos se usan las implementaciones de los paquetes mlxtend y pyECLAT. Mlxtend requiere la creación de tablas con Dummy variables por cada factura, donde cada feature será un binario donde uno indica que hay presencia del producto en la factura y cero que no.

Podemos hacerlo con enteros.

In [44]:
retail_df['Presence'] = 1
basket = pd.pivot_table(retail_df, index='InvoiceNo', columns='StockCode', values='Presence', aggfunc='max', fill_value=0)
basket.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,DCGS0069,DCGS0070,DCGS0076,DCGSSBOY,DCGSSGIRL,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


O con booleanos.

In [45]:
retail_df['Presence'] = True
boolean_basket = pd.pivot_table(retail_df, index='InvoiceNo', columns='StockCode', values='Presence', aggfunc='max', fill_value=False)
boolean_basket.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,DCGS0069,DCGS0070,DCGS0076,DCGSSBOY,DCGSSGIRL,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536366,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536367,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536368,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536369,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


PyECLAT requiere un formato difetente, donde debemos pasar un dataframe en el cual cada fila sea una factura y las columnas contengan los valores de los StockCode comprados.

In [50]:
import numpy as np

transactions = basket.apply(lambda row: [item for item, present in row.items() if bool(present)], axis=1).tolist()
max_len = max(len(t) for t in transactions)
structured_df = pd.DataFrame([t + [np.nan] * (max_len - len(t)) for t in transactions])
structured_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108
0,21730,22752,71053,84029E,84029G,84406B,85123A,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22632,22633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21754,21755,21777,22310,22622,22623,22745,22748,22749,48187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22912,22913,22914,22960,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
print(structured_df.shape, basket.shape)

(19773, 1109) (19773, 3912)
